# Instrument Classification - Part 3 : Track Prediction

In [1]:
import numpy as np
import librosa, os, keras, time
import matplotlib.pyplot as plt

import datetime as dt
import matplotlib.dates as mdates
from matplotlib.collections import PolyCollection

Using TensorFlow backend.


In [5]:
dir = './Instrument-Classification/test/'

instr = ['pia','gac','gel','vio','voi']
model = keras.models.load_model('ICmodel_PGGVV_CQT_20.h5')
thr = [0.5,0.5,0.5,0.5,0.5]
bool = [False,False,False,False,False]

In [6]:
def crop_song(dir,songs):
    predict = []
    even = True
    it = 0
    for f in os.listdir(dir):
        if (it == songs): break
        if (even): 
            even = False 
            continue
        else:
            print(f)
            even = True
            wav,sr = librosa.load(dir+f,mono=True,sr=22050)
            wav = wav / np.sqrt(np.mean(wav**2))
            chunks = []
            for i in range(2*(len(wav)//sr)-1):
                chunks.append(wav[int((i/2)*sr):int((i+2)/2*sr)])
            for ch in chunks:
                mel = librosa.core.cqt(ch,n_bins=96,hop_length=256)
                mel = librosa.core.amplitude_to_db(np.abs(mel))
                predict.append(mel)
        it = it + 1
    return predict

def overlap(prediction):
    new_prediction = []
    for i in range(len(prediction)//2+1):
        y = []
        for j in range(0,len(instr)):
            y.append(0)
        new_prediction.append(y)
    
    for i in range(0,len(prediction),2):
        if (i==0):
            new_prediction[i//2] = 0.5*prediction[i] + 0.5*prediction[i+1]
        elif (i==len(prediction)-1):
            new_prediction[i//2] = 0.5*prediction[i] + 0.5*prediction[i-1]
        else:
            new_prediction[i//2] = 0.25*prediction[i-1] + 0.5*prediction[i] + 0.25*prediction[i+1]
    return new_prediction

def smooth(prediction):
    new_prediction = []
    for i in range(len(prediction)):
        y = []
        for j in range(0,len(instr)):
            y.append(0)
        new_prediction.append(y)
        
    for i in range(len(instr)):
        bool.append(False)

    for i in range(len(prediction)):
        for j in range(len(instr)):
            if i==0:
                if (prediction[i][j] < thr[j] and prediction[i+1][j] > thr[j]): new_prediction[i][j] = prediction[i+1][j]
                elif (prediction[i][j] > thr[j] and prediction[i+1][j] < thr[j]): new_prediction[i][j] = prediction[i+1][j]
                else: new_prediction[i][j] = prediction[i][j]
            elif i==len(prediction)-1:
                if (prediction[i][j] < thr[j] and prediction[i-1][j] > thr[j]): new_prediction[i][j] = prediction[i-1][j]
                elif (prediction[i][j] > thr[j] and prediction[i-1][j] < thr[j]): new_prediction[i][j] = prediction[i-1][j]
                else: new_prediction[i][j] = prediction[i][j]
            else:
                if (prediction[i][j]<thr[j] and prediction[i-1][j]>thr[j] and prediction[i+1][j]>thr[j]):
                    new_prediction[i][j] = (prediction[i-1][j]+prediction[i+1][j])/2
                elif (prediction[i][j]>thr[j] and prediction[i-1][j]<thr[j] and prediction[i+1][j]<thr[j]):
                    new_prediction[i][j] = (prediction[i-1][j]+prediction[i+1][j])/2
                else: new_prediction[i][j] = prediction[i][j]
            if (new_prediction[i][j]> thr[j]): bool[j] = True
    return [new_prediction,bool]

In [7]:
chunks = crop_song(dir,1)
new_chunks = np.expand_dims(np.array(chunks),-1)
prediction = model.predict(new_chunks)

over_prediction = overlap(prediction)
[final_prediction,bool] = smooth(over_prediction)

Official_Lyric_Video-youtube-0NbNZgWYkos-251.wav


In [8]:
print("The track contains: ",end=" ")
for i in range(len(bool)):
    if (bool[i]): print(instr[i],end=" ")

print("\nBegins in 5 seconds")
time.sleep(1)
print("4..",end=" ")
time.sleep(1)
print("3..",end=" ")
time.sleep(1)
print("2..",end=" ")
time.sleep(1)
print("1..",end=" ")
time.sleep(1)
print("Go!")
for i in range(len(final_prediction)):
    time.sleep(1)  
    print(str(i)+"-"+str(i+1)+" sec: ", end =" ")
    for j in range(len(instr)):
        if (final_prediction[i][j]>thr[j]): print(instr[j],round(final_prediction[i][j],2), end =" ")
    print("")

#t = np.arange(0,len(prediction))
#plt.figure(figsize=(50,10))
#plt.plot(t[:150],new_prediction[:150][0],new_prediction[:150][1])

The track contains:  pia gac gel vio voi 
Begins in 5 seconds
4.. 3.. 2.. 1.. Go!
0-1 sec:  
1-2 sec:  
2-3 sec:  vio 0.86 voi 0.5 
3-4 sec:  vio 0.86 voi 0.65 
4-5 sec:  vio 0.98 voi 0.91 
5-6 sec:  vio 0.97 voi 0.96 


KeyboardInterrupt: 

In [81]:
data = [    (dt.datetime(2018, 7, 0, 0, 15), dt.datetime(2018, 7, 0, 0, 30), 'sleep'),
            (dt.datetime(2018, 7, 0, 0, 30), dt.datetime(2018, 7, 0, 0, 45), 'eat'),
            (dt.datetime(2018, 7, 0, 0, 45), dt.datetime(2018, 7, 0, 1, 0), 'work'),
            (dt.datetime(2018, 7, 0, 0, 0), dt.datetime(2018, 7, 0, 1, 30), 'sleep'),
            (dt.datetime(2018, 7, 0, 0, 15), dt.datetime(2018, 7, 0, 1, 30), 'eat'), 
            (dt.datetime(2018, 7, 0, 0, 30), dt.datetime(2018, 7, 0, 1, 45), 'work')
        ]

cats = {"sleep" : 1, "eat" : 2, "work" : 3}
colormapping = {"sleep" : "C0", "eat" : "C1", "work" : "C2"}

verts = []
colors = []
for d in data:
    v =  [(mdates.date2num(d[0]), cats[d[2]]-.4),
          (mdates.date2num(d[0]), cats[d[2]]+.4),
          (mdates.date2num(d[1]), cats[d[2]]+.4),
          (mdates.date2num(d[1]), cats[d[2]]-.4),
          (mdates.date2num(d[0]), cats[d[2]]-.4)]
    verts.append(v)
    colors.append(colormapping[d[2]])

bars = PolyCollection(verts, facecolors=colors)

fig, ax = plt.subplots()
ax.add_collection(bars)
ax.autoscale()
loc = mdates.MinuteLocator(byminute=[0,15,30,45])
ax.xaxis.set_major_locator(loc)
ax.xaxis.set_major_formatter(mdates.AutoDateFormatter(loc))

ax.set_yticks([1,2,3])
ax.set_yticklabels(["sleep", "eat", "work"])
plt.show()

ValueError: day is out of range for month